In [1]:
import numpy as np

import utilities.constants as const
from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.spatial_features import SpatialFeatures as spatial
from features.temporal_features import TemporalFeatures as temporal
from utilities.conversions import Conversions as con
from preprocessing.filters import Filters as filters
import pandas as pd
import time
np.seterr(invalid='ignore')
start = time.time()

In [2]:
%%time

# Reading the geolife dataset and converting to NumPandasTraj.
geolife = pd.read_csv('./data/geolife_sample.csv')
geolife = NumTrajDF(geolife,'lat','lon','datetime','id')

# Reading the gulls dataset and converting to NumPandasTraj.
gulls = pd.read_csv('./data/gulls.csv')
gulls = NumTrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])


# Reading the atlantic dataset, cleaning it up and
# then converting it to NumPandasTraj.
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = NumTrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])
atlantic.drop(columns='Time')
atlantic.drop(columns='Date')

CPU times: user 8.25 s, sys: 118 ms, total: 8.37 s
Wall time: 8.39 s


,,Name,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,Low Wind SE,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,
1851-06-25 00:00:00,AL011851,UNNAMED,0,,HU,28.0,-94.8,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 06:00:00,AL011851,UNNAMED,600,,HU,28.0,-95.4,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 12:00:00,AL011851,UNNAMED,1200,,HU,28.0,-96.0,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 18:00:00,AL011851,UNNAMED,1800,,HU,28.1,-96.5,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 21:00:00,AL011851,UNNAMED,2100,L,HU,28.2,-96.8,80,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-12 12:00:00,AL122015,KATE,1200,,EX,41.3,-50.4,55,981,220,220,180,120,120,120,60,0,0,0,0,0
2015-11-12 18:00:00,AL122015,KATE,1800,,EX,41.9,-49.9,55,983,220,220,180,120,120,120,60,0,0,0,0,0
2015-11-13 00:00:00,AL122015,KATE,0,,EX,41.5,-49.2,50,985,540,520,200,220,120,120,60,0,0,0,0,0


In [3]:
%%time

bbox = filters.get_bounding_box_by_radius(39, 116, 100000)
bbox

CPU times: user 26 µs, sys: 2 µs, total: 28 µs
Wall time: 30.3 µs


(38.100678394081264, 114.84275815636957, 39.89932160591873, 117.15724184363044)

In [4]:
bbox_filt = filters.filter_by_bounding_box(geolife, bbox, True)
print(f"Original length of geolife: {len(geolife)}")
print(f"Filtered length of geolife: {len(bbox_filt)}")

Original length of geolife: 217653
Filtered length of geolife: 356
